<a href="https://colab.research.google.com/github/harikrishnankh27/Analyzing-Retail-Queues-Using-Computer-Vision_yolo/blob/main/dl_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install inference supervision

In [ ]:
import inference
import supervision as sv

In [ ]:
from inference.models.utils import get_roboflow_model

# Load the YOLOv8 model
model = get_roboflow_model(model_id="yolov8n-640")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2
import numpy as np
import csv
tracker = sv.ByteTrack()


# Open the video file
video_path = "/content/drive/MyDrive/dl_pro/Retail.mp4"
cap = cv2.VideoCapture(video_path)

fps = 20

# Define the output video path
output_video_path = "/content/drive/MyDrive/dl_pro/output_video.mp4"

# Define the codec and create a VideoWriter object for the output video
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (frame_width, frame_height))

# Define ROI (Region of Interest) coordinates for the rectangular regions
roi1_coords = np.array([[747, 622],[707, 38],[807, 22],[931, 654],[747, 622]], dtype=np.int32)

roi2_coords = np.array([[1039, 62],[1243, 546],[1271, 502],[1231, 286],[1107, 34],[1039, 62]], dtype=np.int32)

people_enter_queue = {}

timespent = []

filename = "queue_time.csv"

# Open the file in write mode
file = open(filename, 'w', newline='')

# Create a CSV writer object
csv_writer = csv.writer(file)

frame_count =0

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        annotated_frame = frame.copy()

        #draw the ROIs
        cv2.drawContours(annotated_frame, [roi1_coords], -1, (255, 0, 0), 3)
        cv2.drawContours(annotated_frame, [roi2_coords], -1, (255, 0, 0), 3)

        # Run YOLOv8 tracking on the original frame
        results = model.infer(frame)
        # print(results[0].predictions)
        detections = sv.Detections.from_inference(results[0])
        detections = tracker.update_with_detections(detections)
        # print(detections)

        # Get the boxes and track IDs
        boxes = detections.xyxy
        #print(boxes)

        print("Found: ", people_enter_queue)
        # print(type(detections.tracker_id))
        if type(detections.tracker_id) == np.ndarray:
          # track_ids = results[0].boxes.id.int().cpu().tolist()
          track_ids = detections.tracker_id

          # Check if centre of bounding boxes is inside the ROI
          for box, track_id in zip(boxes, track_ids):

              print("Tracking:",track_id)

              x1, y1, x2, y2 = box
              x1, y1, x2, y2= int(x1), int(y1), int(x2), int(y2)

              # print(x1,y1,x2,y2)
              x = (x1+x2)/2
              y = (y1+y2)/2

              # Visualize the people being tracked in queues on the frame
              if ((cv2.pointPolygonTest(roi1_coords, (x,y), False) > 0) or ((cv2.pointPolygonTest(roi2_coords, (x,y), False)) > 0)):
                  if (str(track_id) not in people_enter_queue):
                    #get timestamp
                    people_enter_queue[str(track_id)] = str(frame_count)

                  cv2.rectangle(annotated_frame, (x1,y1), (x2, y2), (0,255,0), 2)
                  cv2.putText(annotated_frame,"Person id:"+ str(track_id), (x1,y1-5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (20,255,0), 2)
              else:
                print("outside:",track_id)

                if (str(track_id) in people_enter_queue):
                  #get timestamp
                  exit = frame_count

                  #get first timestamp
                  start = people_enter_queue[str(track_id)]

                  time_spent = (exit - int(start))/fps
                  print("time spent ", time_spent, "by person", track_id)

                  timespent.append(time_spent)

                  # Write string to the file
                  csv_writer.writerow(["Time spent by person "+ str(track_id)+" in line is "+str(time_spent)])

                  people_enter_queue.pop(str(track_id))

        # Write the frame to the output video
        out.write(annotated_frame)
        frame_count = frame_count+1

    else:

      #for ppl still in line at the end of the video
      for person in people_enter_queue:

        #get timestamp
        exit = frame_count

        #get first timestamp
        start = people_enter_queue.get(person)

        time_spent = (exit - int(start))/fps
        print("time spent ", time_spent, "by person", person)

        timespent.append(time_spent)

        # Write string to the file
        csv_writer.writerow(["Time spent by person "+ str(person)+" in line is "+str(time_spent)])

      average = sum(timespent)/len(timespent)

      print("Average of list: ", round(average,3))

      csv_writer.writerow(["Average time spent in line is "+str(round(average,3))])
        # Break the loop if the end of the video is reached
      break


# Release the video capture object, release the output video, and close the display window
cap.release()
out.release()
cv2.destroyAllWindows()
file.close()

print(f"Output video saved at: {output_video_path}")


Found:  {}
Tracking: 67
Tracking: 68
outside: 68
Tracking: 69
Tracking: 70
outside: 70
Found:  {'67': '0', '69': '0'}
Tracking: 67
Tracking: 68
outside: 68
Tracking: 69
Tracking: 70
outside: 70
Found:  {'67': '0', '69': '0'}
Tracking: 67
Tracking: 68
outside: 68
Tracking: 69
Tracking: 70
outside: 70
Found:  {'67': '0', '69': '0'}
Tracking: 67
Tracking: 68
outside: 68
Tracking: 69
Tracking: 70
outside: 70
Found:  {'67': '0', '69': '0'}
Tracking: 67
Tracking: 68
outside: 68
Tracking: 69
Tracking: 70
outside: 70
Found:  {'67': '0', '69': '0'}
Tracking: 67
Tracking: 68
outside: 68
Tracking: 69
Tracking: 70
outside: 70
Found:  {'67': '0', '69': '0'}
Tracking: 67
Tracking: 68
outside: 68
Tracking: 69
Tracking: 70
outside: 70
Found:  {'67': '0', '69': '0'}
Tracking: 67
Tracking: 68
outside: 68
Tracking: 69
Tracking: 70
outside: 70
Found:  {'67': '0', '69': '0'}
Tracking: 67
Tracking: 68
outside: 68
Tracking: 69
Tracking: 70
outside: 70
Found:  {'67': '0', '69': '0'}
Tracking: 67
Tracking: 68


In [ ]:
import pandas as pd
df = pd.read_csv("queue_time.csv")
df

,Time spent by person 25 in line is 4.2
0,Time spent by person 23 in line is 8.3
1,Time spent by person 28 in line is 6.55
2,Average time spent in line is 6.35
